<a href="https://colab.research.google.com/github/VonRottweiler/Recidivism/blob/main/XgBoosting_Model.v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
#lightgbm, catboost, datarobot

import pandas as pd
import numpy as np
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

test = pd.read_csv('test.csv')

data = pd.read_csv('train.csv')

from sklearn.impute import SimpleImputer
mean = SimpleImputer(strategy='mean')
mode = SimpleImputer(strategy='most_frequent')

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

col = 0
for col in range(1, len(data.columns)):
  if (data[[data.columns[col]][0]].dtypes) == object:
    mode.fit(data[[data.columns[col]]])
    data[[data.columns[col]]] = mode.transform(data[[data.columns[col]]])
    data[data.columns[col]] = le.fit_transform(data[data.columns[col]])
  elif (data[[data.columns[col]][0]].dtypes) == float:
    mean.fit(data[[data.columns[col]]])
    data[[data.columns[col]]] = mean.transform(data[[data.columns[col]]])

col = 0
for col in range(1, len(test.columns)):
  if (test[[test.columns[col]][0]].dtypes) == object:
    mode.fit(test[[test.columns[col]]])
    test[[test.columns[col]]] = mode.transform(test[[test.columns[col]]])
    test[test.columns[col]] = le.fit_transform(test[test.columns[col]])
  elif (test[[test.columns[col]][0]].dtypes) == float:
    mean.fit(test[[test.columns[col]]])
    test[[test.columns[col]]] = mean.transform(test[[test.columns[col]]])

GxBoosting Training and Model Testing

In [13]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(data.drop('Recidivism_Arrest', axis=1), data['Recidivism_Arrest'], test_size=0.2, random_state=42)

# Define the XGBoost model and set hyperparameters
acc = 0
eval_set = [(X_test, y_test)]
for n_estimators in [500,600,700]: # these are just examples, need to expand this list in practice
  for learning_rate in [.05,.01,.15]:
    for max_depth in [8, 9, 10]:
      model = xgb.XGBClassifier(objective='binary:logistic', learning_rate=learning_rate, max_depth=max_depth, n_estimators=n_estimators, subsample=0.8, colsample_bytree=0.8, seed=42, eval_metric="logloss")
      model.fit(X_train, y_train, eval_set=eval_set, verbose=False)

      y_pred = model.predict(X_test)
      acc_temp=sklearn.metrics.accuracy_score(y_test,y_pred)
      if(acc_temp>acc):
        acc=acc_temp
        print('Accuracy: {:.4f}'.format(accuracy_score(y_test, y_pred)))
        print([n_estimators,learning_rate,max_depth,acc])

Accuracy: 0.6802
[500, 0.05, 8, 0.6802197802197802]
Accuracy: 0.6868
[500, 0.05, 9, 0.6868131868131868]
Accuracy: 0.6885
[600, 0.01, 9, 0.6884615384615385]


GxBoosting Test Data Predictions

In [19]:
xgmodel = xgb.XGBClassifier(objective='binary:logistic', learning_rate=.01, max_depth=9, n_estimators=600, subsample=0.8, colsample_bytree=0.8, seed=42, eval_metric="logloss")
xgmodel.fit((data.drop('Recidivism_Arrest', axis=1)), data['Recidivism_Arrest'], verbose=False)

y_pred = xgmodel.predict(test)
test['Recidivism_Arrest']=y_pred
test[['ID','Recidivism_Arrest']].to_csv('submission.gxboosting.csv',index=False)

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
